1. Showing a sample of the data to an AI model like ChatGPT
2. Asking for the Python code for the analysis
3. Inspecting and running the code
4. Automating it locally in our machines to run forever

In [7]:
import pandas as pd

df = pd.read_csv("../assets/stock-trading-data.csv")

df

,Date,Stock Symbol,Open Price,Close Price,Volume,Moving Average (50-day),RSI (Relative Strength Index)
0,2024-01-01,AAPL,249.816048,240.444631,768234,240.444631,0.000000
1,2024-01-02,AAPL,480.285723,483.013931,478480,361.729281,100.000000
2,2024-01-03,AAPL,392.797577,389.084696,891971,370.847753,72.086287
3,2024-01-04,AAPL,339.463394,339.634808,601157,363.044516,62.850185
4,2024-01-05,AAPL,162.407456,170.558786,4478727,324.547370,43.704255
...,...,...,...,...,...,...,...
495,2024-05-13,TSLA,288.612984,286.678211,4032441,322.118171,45.674494
496,2024-05-14,TSLA,317.293122,311.704345,1307068,318.940871,43.853621
497,2024-05-15,TSLA,163.698295,162.066291,4464406,318.721371,39.550593
498,2024-05-16,TSLA,181.480500,175.552065,494274,314.121417,37.937391


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import os

# Load data
def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df.dropna(inplace=True)  # Handle missing values
    df.sort_values(by=['Date'], inplace=True)
    return df

# Compute daily returns and rolling volatility
def compute_returns_volatility(df):
    df['Daily Return'] = df.groupby('Stock Symbol')['Close Price'].pct_change()
    df['Rolling Volatility'] = df.groupby('Stock Symbol')['Daily Return'].rolling(window=20).std().reset_index(level=0, drop=True)
    return df

# Compute SMA and EMA
def compute_moving_averages(df):
    df['SMA_20'] = df.groupby('Stock Symbol')['Close Price'].transform(lambda x: x.rolling(window=20).mean())
    df['EMA_20'] = df.groupby('Stock Symbol')['Close Price'].transform(lambda x: x.ewm(span=20, adjust=False).mean())
    return df

# Compute Bollinger Bands
def compute_bollinger_bands(df):
    rolling_mean = df.groupby('Stock Symbol')['Close Price'].transform(lambda x: x.rolling(window=20).mean())
    rolling_std = df.groupby('Stock Symbol')['Close Price'].transform(lambda x: x.rolling(window=20).std())
    df['Upper Band'] = rolling_mean + (rolling_std * 2)
    df['Lower Band'] = rolling_mean - (rolling_std * 2)
    return df

# Compute MACD
def compute_macd(df):
    df['EMA_12'] = df.groupby('Stock Symbol')['Close Price'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
    df['EMA_26'] = df.groupby('Stock Symbol')['Close Price'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['Signal Line'] = df.groupby('Stock Symbol')['MACD'].transform(lambda x: x.ewm(span=9, adjust=False).mean())
    return df

# Generate visualizations
def plot_stock_prices(df, symbol):
    stock_df = df[df['Stock Symbol'] == symbol]
    plt.figure(figsize=(12, 6))
    plt.plot(stock_df['Date'], stock_df['Close Price'], label='Close Price')
    plt.plot(stock_df['Date'], stock_df['SMA_20'], label='20-day SMA', linestyle='dashed')
    plt.plot(stock_df['Date'], stock_df['EMA_20'], label='20-day EMA', linestyle='dotted')
    plt.fill_between(stock_df['Date'], stock_df['Upper Band'], stock_df['Lower Band'], color='gray', alpha=0.3, label='Bollinger Bands')
    plt.title(f'{symbol} Stock Price Analysis')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid()
    plt.savefig(f'{symbol}_stock_price.png')
    plt.close()

# Generate heatmap of correlations
def plot_correlation_heatmap(df):
    numeric_cols = ['Close Price', 'Daily Return', 'Rolling Volatility', 'SMA_20', 'EMA_20', 'RSI (Relative Strength Index)', 'MACD', 'Signal Line']
    correlation_matrix = df[numeric_cols].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Stock Metric Correlation Heatmap')
    plt.savefig('correlation_heatmap.png')
    plt.close()

# Export analysis to Excel
def export_to_excel(df):
    output_file = 'financial_analysis_report.xlsx'
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='Stock Data', index=False)
        workbook = writer.book
        worksheet = writer.sheets['Stock Data']
        worksheet.insert_image('J2', 'AAPL_stock_price.png')
        worksheet.insert_image('J20', 'correlation_heatmap.png')
    print(f'Report saved as {output_file}')

# Main function



df = compute_returns_volatility(df)
df = compute_moving_averages(df)
df = compute_bollinger_bands(df)
df = compute_macd(df)

plot_stock_prices(df, 'AAPL')
# plot_correlation_heatmap(df)
# export_to_excel(df)

In [ ]:
![](./)